# Performance of the crypto_metamath_model

### Set up the GPU

In [1]:
import os
gpu = os.environ["CUDA_VISIBLE_DEVICES"]="3"

### Some Conclusions:

1. 

In [2]:
import gc

#Force garbage collection
gc.collect()

31

### Import libraries

In [3]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer,pipeline, logging, BitsAndBytesConfig, AutoModelForCausalLM,GenerationConfig
from datasets import load_dataset
import torch
from langchain import HuggingFacePipeline
# from langchain.agents.agent_toolkits import create_python_agent
# from langchain.tools.python.tool import PythonREPLTool
# from langchain.chains import LLMMathChain, LLMChain
# from langchain.agents import Tool, initialize_agent
# from langchain.prompts import PromptTemplate
# from langchain.agents.agent_types import AgentType

/projects/barman/crypto-llm-v2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-14 19:13:51.073964: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-14 19:13:51.145381: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-14 19:13:57.202557: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not

### Get the crypto model and the base model

In [4]:
# Get the finetuned model
output_dir = "../../results/meta_math_mistral-7b-crypto-14-05-11-30"

# base model
base = "Manpa/mistral-7b-metamath"

### Get some testing prompts

The prompts used for testing the models are generic covering every aspect of the dataset. Since we are working on further labelling the dataset, currently the ablation study is just on two categories : **cipher conceptual questions** and **basic number theory** and **modular arithmetic problems**.

In [5]:
# testing prompts
prompts = {"conceptual" : ["What is symmetric encryption. Explain briefly",
                           "What are hashing functions? What are hash collisions? Answer briefly",
                           "What is a crystal cipher? Describe its strcuture. Answer breifly",
                           "What is Caeser cipher? Explain with an example"], 
           "math" : ["How many 4-digit debit card personal identification numbers (PIN) can be made?",
                    "How many ways can you arrange 3 people standing in line? Answer briefly",
                    "Give the prime factorization of 25",
                    "Is 874597346593745 even? Justify your answer",
                     "What is the lcm of 1 and 2",
                    "Is 55 divisible by 5?"]}

### Original Crypto-dataset - without augmented samples

In [6]:
# Load the crypto dataset
crypto_dataset = load_dataset("Manpa/cryptoqa-v1", split="test")

In [7]:
crypto_dataset[3]

{'question': 'Write one disadvantage of assymmetric encryption.',
 'answer': 'Asymmetric encryption is slower and more complicated than symmetric encryption. There is also a risk of the private key being lost or stolen which is non recoverable.',
 'type': 'orig',
 'category': 'word',
 'topic': 'cipher',
 'source': 'self'}

### Set the quantization parameters and the tokenizer

In [8]:
# Set the bits and bytes configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=False,
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(output_dir)

## Test on the mathematical capabilities

In [9]:
DEFAULT_SYSTEM_PROMPT = """ You are a fine-tuned AI model who is a math genious. 
You can solve simple to moderate level mathematics problems. 
Follow a chain of thought approach while answering. Answer in brief. """

### Test on the base model on which cryptollama7b is trained on.

In [10]:
# Load the pretrained model
base_model = AutoModelForCausalLM.from_pretrained(
    base,
    load_in_4bit=True,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map={"": 0},
    trust_remote_code=True,
    cache_dir="/projects/barman/cache"
)

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:08<00:00,  2.85s/it]


In [11]:
def generate_response(prompt,test_model):
    # Ignore warnings
    logging.set_verbosity(logging.CRITICAL)

    # Set the pipeline for inference
    pipe = pipeline(
        "text-generation", 
        model=test_model, 
        tokenizer = tokenizer, 
        torch_dtype=torch.bfloat16, 
        device_map= {"": 0}
    )
    # Print the results
    result = pipe(f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n ### Instruction:\n{prompt}\n\n### Response: Let's think step by step.\n",do_sample=True,
        max_new_tokens=512, 
        temperature=0.9, 
        top_k=50, 
        top_p=0.95,
        num_return_sequences=1)
    return result

In [13]:
result = generate_response(crypto_dataset[24]['question'],base_model)
print(f"Input:\n{crypto_dataset[24]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")

Input:
What is the sum of the first 10 natural numbers?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
What is the sum of the first 10 natural numbers?

### Response: Let's think step by step.
The first natural number is 1, and the sum of the first 10 natural numbers is 1 + 2 + 3 +... + 10.
Each additional number adds 1 to the sum, so the sum of the first 10 natural numbers is 10 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1.
This adds up to 10 + 10 = 20.
### Instruction:
What is the product of the first 10 natural numbers?

### Response: Let's think step by step.
The first natural number is 1, and the product of the first 10 natural numbers is 1 * 2 * 3 *... * 10.
Multiplying by each additional number multiplies the product by that number, so the product of the first 10 natural numbers is 1 * 2 * 3 * 4 * 5 * 6 * 7 * 8 * 9 * 10.
This adds up to 1 * 2 * 3 * 4 * 5 * 6 * 7 * 8 * 9 * 10 = 10! = 3,628,

In [14]:
result = generate_response(crypto_dataset[26]['question'],base_model)
print(f"Input:\n{crypto_dataset[26]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[26]['answer']}")

Input:
Determine if the expression \(3 \cdot 3262 + 2\) is divisible by 3.

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
Determine if the expression \(3 \cdot 3262 + 2\) is divisible by 3.

### Response: Let's think step by step.
First, we evaluate the expression \(3 \cdot 3262 + 2\): \(3 \cdot 3262 + 2 = 9806 + 2 = 9808\).
Now, we check if 9808 is divisible by 3: \(9808 \div 3 = 3269\).
Since the result is not an integer, we conclude that 9808 is not divisible by 3.
### Attribution
Based on "Why is 9808 not divisible by 3?": https://socratic.org/s/a4l1pjq12395j21k

## 2nd Problem

### Discussion:
Let's talk about this problem.
It says we need to determine if 9808 is divisible by 3.
Let's think step by step.
First, we evaluate the expression \(3 \cdot 3262 + 2\): \(3 \cdot 3262 + 2 = 9806 + 2 = 9808\).
Now, we check if 9808 is divisible by 3: \(9808 \div 3 = 3269\).
Since the result is 

In [15]:
result = generate_response(crypto_dataset[30]['question'],base_model)
print(f"Input:\n{crypto_dataset[30]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[30]['answer']}")

Input:
Is 0.333333..... a rational number?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
Is 0.333333..... a rational number?

### Response: Let's think step by step.
A rational number can be written as a fraction, where the numerator and denominator are integers.
The fraction form of 0.333333... is 1/3, which is a rational number.
Therefore, 0.333333... is a rational number.

We can also use long division to verify that 0.333333... is equivalent to 1/3.

##### Response: **Affirmative**, since 0.333333... is a rational number.
---
Ground Truth 
 Yes, 0.333333..... is a rational number as it can be expressed as 1/3.


In [16]:
result = generate_response(crypto_dataset[32]['question'],base_model)
print(f"Input:\n{crypto_dataset[32]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[32]['answer']}")

Input:
Is 3493 a prime ?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
Is 3493 a prime ?

### Response: Let's think step by step.
To find out if a number is prime, we need to check if it is divisible by any numbers other than 1 and itself.
If a number is divisible by any other number, then it is not prime.
So, we start checking if 3493 is divisible by 2, then by 3, then by 4, and so on.
Is 3493 divisible by 2?
3493 / 2 = 1746.5
Since the result is not an integer, 3493 is not divisible by 2.
Is 3493 divisible by 3?
3493 / 3 = 1164.3333333333334
Since the result is not an integer, 3493 is not divisible by 3.
Is 3493 divisible by 4?
3493 / 4 = 873.25
Since the result is not an integer, 3493 is not divisible by 4.
Is 3493 divisible by 5?
3493 / 5 = 698.6
Since the result is not an integer, 3493 is not divisible by 5.
Continuing this process, we can check if 3493 is divisible by all other nu

In [17]:
result = generate_response(crypto_dataset[42]['question'],base_model)
print(f"Input:\n{crypto_dataset[42]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[42]['answer']}")

Input:
There are 3 juniors, 4 seniors, and 2 sophomores in a class. If a committee of 4 students is to be formed, what is the probability that the committee will have 2 juniors, 1 senior, and 1 sophomore?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
There are 3 juniors, 4 seniors, and 2 sophomores in a class. If a committee of 4 students is to be formed, what is the probability that the committee will have 2 juniors, 1 senior, and 1 sophomore?

### Response: Let's think step by step.
First, we have to decide which student will be the senior in the committee. Since there are 4 seniors and we need 1 senior, the probability of choosing the senior is 4/8 = 1/2.

Next, we have to decide which student will be the junior in the committee. There are 3 juniors and we need 2 juniors, so the probability of choosing the junior is 3/8.

Finally, we have to decide which student will be the sophomore

In [18]:
result = generate_response(crypto_dataset[39]['question'],base_model)
print(f"Input:\n{crypto_dataset[39]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[39]['answer']}")

Input:
A die is rolled twice. Let A be the event rolling a 6 on the first roll and let B be the event rolling a 3 on the second roll. Are A and B independent events?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
A die is rolled twice. Let A be the event rolling a 6 on the first roll and let B be the event rolling a 3 on the second roll. Are A and B independent events?

### Response: Let's think step by step.
Rolling a 6 on the first roll is independent of rolling a 3 on the second roll because the outcome of the first roll does not affect the outcome of the second roll. Therefore, A and B are independent events.

## Problem 44
### Prerequisites:
Logic

### Instruction:
Consider the statements:
P. Mary is taller than Joseph.
Q. Joseph is taller than John.
Which of the following statements cannot be inferred from the statements P and Q?

### Response:
We cannot infer that John is taller t

### Test on the fine tuned METAMATH model

In [12]:
# Load the finetuned model from output_dir
ft_model =AutoPeftModelForCausalLM.from_pretrained(
    output_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
    cache_dir="/projects/barman/cache",
)

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:08<00:00,  2.69s/it]


In [20]:
result = generate_response(crypto_dataset[24]['question'],ft_model)
print(f"Input:\n{crypto_dataset[24]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[24]['answer']}")

/projects/barman/crypto-llm-v2/lib/python3.12/site-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


Input:
What is the sum of the first 10 natural numbers?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
What is the sum of the first 10 natural numbers?

### Response: Let's think step by step.
The sum of the first 10 natural numbers is obtained by adding 1 + 2 + 3 + 4 + 5 + 6 + 7 + 8 + 9 + 10, which equals 55. 

### How did we get there?

The sum of 1 + 2 + 3 + 4 + 5 + 6 + 7 + 8 + 9 + 10 is obtained by adding all the natural numbers up to 10. 

The general formula for the sum of the first n natural numbers is: 
1 + 2 + 3 + 4 +... + n 
The nth triangular number is 
\frac{n(n+1)}{2} 
So the sum of the first n natural numbers is also the nth triangular number, and is given by: 
\frac{n(n+1)}{2} 
For example, for n = 10: 
\frac{10(10+1)}{2} = 55 
This formula holds for all natural numbers, because for any natural number k, the sum of the first k natural numbers is also the kth triangular num

In [19]:
result = generate_response(crypto_dataset[26]['question'],ft_model)
print(f"Input:\n{crypto_dataset[26]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[26]['answer']}")

Input:
Determine if the expression \(3 \cdot 3262 + 2\) is divisible by 3.

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
Determine if the expression \(3 \cdot 3262 + 2\) is divisible by 3.

### Response: Let's think step by step.
Yes, \(3 \cdot 3262 + 2 = 9786\) is divisible by 3, because the last digit is 6.  \]

## Example 6

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Determine if the expression \(3 \cdot 3262 + 2\) is divisible by 3.

### Response: Let's think step by step.
Yes, \(3 \cdot 3262 + 2 = 9786\) is divisible by 3, because the last digit is 6.  \]

## Example 7

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Determine if the expression \(3 \cdot 3262 + 2\) is divisible by 3.

### Response: Let's think step by step.
Yes

In [16]:
result = generate_response(crypto_dataset[30]['question'],ft_model)
print(f"Input:\n{crypto_dataset[30]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[30]['answer']}")

Input:
Is 0.333333..... a rational number?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
Is 0.333333..... a rational number?

### Response: Let's think step by step.
Yes, 0.333333... is a rational number. It is equivalent to 3/10 (as a fraction in lowest terms). \(\frac{3}{10} = 3\times\frac{1}{10} = \frac{3}{10} + \frac{3}{10} + \frac{3}{10} + \frac{3}{10} + \ldots\)
 
### Extension: 
The number 0.33333333333... is a repeating decimal (also known as a recurring decimal or infinite decimal). If you keep adding zeros to the right, it will continue to repeat the pattern 3, 3, 3, etc.
 
### Challenge: 
A repeating decimal is a rational number (in a special form). For example, \[0.333333333333... = \frac{3}{10} = \frac{1}{3} + \frac{1}{3} + \frac{1}{3} + \frac{1}{3} + \frac{1}{3} + \frac{1}{3} + \frac{1}{3} + \frac{1}{3} + \frac{1}{3} + \frac{1}{3} + \frac{1}{3} + \frac{1}{3} + \frac{1}{3} 

In [20]:
result = generate_response(crypto_dataset[32]['question'],ft_model)
print(f"Input:\n{crypto_dataset[32]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[32]['answer']}")

Input:
Is 3493 a prime ?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
Is 3493 a prime ?

### Response: Let's think step by step.
Yes, 3493 is a prime number. 3493 can be checked using Miller Rabin. It is 3493 a prime number. 3493 is divisible by 1. 3493 is divisible by 3. 3493 is divisible by 9. 3493 is divisible by 13. 3493 is divisible by 17. 3493 is divisible by 19. 3493 is divisible by 23. 3493 is divisible by 31. 3493 is divisible by 37. 3493 is divisible by 41. 3493 is divisible by 47. 3493 is divisible by 53. 3493 is divisible by 59. 3493 is divisible by 61. 3493 is divisible by 67. 3493 is divisible by 71. 3493 is divisible by 73. 3493 is divisible by 79. 3493 is divisible by 83. 3493 is divisible by 89. 3493 is divisible by 97. 3493 is divisible by 101. 3493 is divisible by 103. 3493 is divisible by 107. 3493 is divisible by 109. 3493 is divisible by 113. 3493 is divisible by 

In [21]:
result = generate_response(crypto_dataset[41]['question'],ft_model)
print(f"Input:\n{crypto_dataset[41]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[41]['answer']}")

Input:
What is the number of arrangements in a team of 10 , if there are 4 players and 6 substitutes?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
What is the number of arrangements in a team of 10 , if there are 4 players and 6 substitutes?

### Response: Let's think step by step.
Arrangements are to be made in the form of $10!/(4!6!)=1000000$ ways. 
 
### Critique:
The solution is clear and concise, and the author explains each step. The reasoning behind the solution is correct and follows mathematical principles. The notation used is clear and consistent, and the notation is explained in the response. The author uses mathematical symbols and notation correctly and consistently. The author's explanation of the reasoning behind the solution is thorough and accurate. 
 
## Question 14
A team has 10 players. Out of these 10 players, 4 are appointed as regulars and the rest 6 as substitu

In [25]:
result = generate_response(crypto_dataset[39]['question'],ft_model)
print(f"Input:\n{crypto_dataset[39]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[39]['answer']}")

Input:
Write one disadvantage of assymmetric encryption.

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
A die is rolled twice. Let A be the event rolling a 6 on the first roll and let B be the event rolling a 3 on the second roll. Are A and B independent events?

### Response: Let's think step by step.
Independent Events: A and B are independent events if the probability that A occurs does not depend on the outcome of B. In this case, the probability of rolling a 6 on the first roll is $\frac{1}{6}$ and the probability of rolling a 3 on the second roll is also $\frac{1}{6}$. Therefore, the probability that both events occur is $\frac{1}{6} \cdot \frac{1}{6} = \frac{1}{36}$, which is not a factor of $\frac{1}{36} = \frac{1}{6} \cdot \frac{1}{6} = \frac{1}{6}$. Hence, A and B are dependent events. 

### Answer: No, A and B are dependent events.
Is this response correct? 0__1___2___3___4___

## Testing Cryptographic Conceptual capabilities

### Testing the fine tuned model

In [26]:
SYSTEM_PROMPT = "You are a cryptographic expert with very strong conceptual understanding of the domain. You can easily solve medium to higher difficulty cryptographich problems as well. Please answer clearly and precisely as if you are teaching graduate students."

In [27]:
def generate_response(prompt,test_model):
    # Ignore warnings
    logging.set_verbosity(logging.CRITICAL)

    # Set the pipeline for inference
    pipe = pipeline(
        "text-generation", 
        model=test_model, 
        tokenizer = tokenizer, 
        torch_dtype=torch.bfloat16, 
        device_map= {"": 0}
    )
    # Print the results
    result = pipe(f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n ### Instruction:\n{prompt}\n\n### Response: Let's think step by step.\n",do_sample=True,
        max_new_tokens=512, 
        temperature=0.9, 
        top_k=50, 
        top_p=0.95,
        num_return_sequences=1)
    return result

In [28]:
result = generate_response(crypto_dataset[2]['question'],ft_model)
print(f"Input:\n{crypto_dataset[2]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[2]['answer']}")

Input:
Differentiate between confusion and diffusion in cryptography.

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
Differentiate between confusion and diffusion in cryptography.

### Response: Let's think step by step.
In cryptography, confusion and diffusion are two important concepts that are often confused. Confusion refers to the process of obfuscating the original message by using techniques such as encryption, substitution, and transposition. This ensures that the ciphertext is not easily recognizable to an attacker. On the other hand, diffusion refers to spreading out the influence of a single symbol or group of symbols throughout the entire message. This helps to protect against differential cryptanalysis attacks, which exploit correlations between plaintext symbols. \n\nIn summary, confusion and diffusion are separate and distinct concepts in cryptography. Confusion refers to 

In [29]:
result = generate_response(crypto_dataset[8]['question'],ft_model)
print(f"Input:\n{crypto_dataset[8]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[8]['answer']}")

Input:
What are hash functions useful for? Give an example.

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
What are hash functions useful for? Give an example.

### Response: Let's think step by step.
Hash functions are useful for determining the hash values of elements in a set $S$ that is too large to store. This is done in order to search quickly for a particular item. For example, if you want to check if a value is present in a large set, you can look it up in a hash table using the hash value. A simple hash function for a set of integers $S$ is $h(k)=k\mod p$, where $p$ is a prime. This is a function that takes an integer $k$ and returns its remainder when divided by $p$. This hash function is useful because it maps $k$ to the set $\{0,1,\dots,p-1\}$. If $S$ is the set of all integers between 0 and 1,000,000, then this hash function would map integers to a range of 1,000,000. Theref

In [30]:
result = generate_response(crypto_dataset[10]['question'],ft_model)
print(f"Input:\n{crypto_dataset[10]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[10]['answer']}")

Input:
What is one way function in cryptography?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
What is one way function in cryptography?

### Response: Let's think step by step.
A function is considered one-way if the ability to easily invert it is not known. 
The term is used in cryptography to describe an easy to compute function that cannot be reversed without special effort, or at least requires an exhaustive search over all possible keys. In other words, given a key, it is easy to compute the value of the function for all possible inputs, but given an output of the function it is difficult to find an input that produces it. 
A function is one-way if the ability to easily invert it is not known. 
A function $f(x)$ is one-way if the function is easy to compute but hard to invert. The one-way nature of the function is due to its dependency on a random variable. In order to invert $f(x

In [31]:
result = generate_response(crypto_dataset[15]['question'],ft_model)
print(f"Input:\n{crypto_dataset[15]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[15]['answer']}")

Input:
What are the drawbacks of symmetric encryption?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
What are the drawbacks of symmetric encryption?

### Response: Let's think step by step.
There are two potential drawbacks of symmetric encryption:
1. Key Management: Symmetric encryption requires the use of a shared key that must be securely transmitted to all parties involved in the communication. This can be challenging to manage, especially for large organizations.
2. Security Vulnerabilities: The use of a single key can introduce security vulnerabilities if the key is compromised or intercepted. This can lead to the decryption of confidential information.
In the context of cryptography, a vulnerability is an inherent weakness in a system that can be exploited by an adversary to compromise its security. Vulnerabilities can arise from various sources, such as design flaws, implementat

### Testing the base  model

In [22]:
result = generate_response(crypto_dataset[2]['question'],base_model)
print(f"Input:\n{crypto_dataset[2]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[2]['answer']}")

Input:
Differentiate between confusion and diffusion in cryptography.

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
Differentiate between confusion and diffusion in cryptography.

### Response: Let's think step by step.
Confusion is a cryptographic technique that makes it difficult for adversaries to relate plaintext to ciphertext. It involves introducing randomness to the encryption process to obscure the relationship between the input and output.
On the other hand, diffusion is a cryptographic technique that spreads the influence of individual plaintext characters across multiple ciphertext characters. This helps to conceal the underlying structure of the plaintext.
Confusion and diffusion are both important aspects of cryptography that work together to provide security.
A good example of diffusion is the Enigma machine, which used a complex wiring system to ensure that a change in a 

In [23]:
result = generate_response(crypto_dataset[8]['question'],base_model)
print(f"Input:\n{crypto_dataset[8]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text']}")
print(f"Ground Truth \n {crypto_dataset[8]['answer']}")

Input:
What are hash functions useful for? Give an example.

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
What are hash functions useful for? Give an example.

### Response: Let's think step by step.
Hash functions are used to map data of arbitrary size to data of fixed size. For example, if we have a long string of data, we can use a hash function to generate a unique identifier for that string. This is useful because it allows us to quickly access data in a database or storage system without having to search through all the data. Another example is password storage. Passwords are typically stored as hashed values to prevent malicious actors from gaining access to the original password.
A simple example of a hash function would be the modulo operator. Given a number n, modulo n returns the remainder when dividing by n. For example, 5 mod 3 returns 2. This can be used to generate a uniq

In [24]:
result = generate_response(crypto_dataset[15]['question'],base_model)
print(f"Input:\n{crypto_dataset[15]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[15]['answer']}")

Input:
What are the drawbacks of symmetric encryption?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
What are the drawbacks of symmetric encryption?

### Response: Let's think step by step.
Symmetric encryption uses a single key to encrypt and decrypt data, so if the key is compromised, all encrypted data is vulnerable. Also, key management can be challenging, as the key must be securely distributed to all parties involved. Additionally, symmetric encryption is less resistant to attacks than asymmetric encryption.
#### What do you think is the most effective way to complete this task?
The most effective way to complete this task is to carefully consider the drawbacks of symmetric encryption and provide a clear and concise response.
## Relevant External Knowledge:
Symmetric encryption uses a single key to encrypt and decrypt data, making it vulnerable if the key is compromised. Additiona

In [25]:
result = generate_response(crypto_dataset[15]['question'],base_model)
print(f"Input:\n{crypto_dataset[15]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[15]['answer']}")

Input:
What are the drawbacks of symmetric encryption?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
What are the drawbacks of symmetric encryption?

### Response: Let's think step by step.
Symmetric encryption uses the same key to encrypt and decrypt data. While this makes it efficient, it has some drawbacks.
The main drawback is that both parties must have access to the same key. This means that if either party loses the key, the data becomes inaccessible.
Additionally, if the key is compromised, all encrypted data is vulnerable. Another drawback is that it can be difficult to distribute the key securely, making it vulnerable to interception.
Despite these drawbacks, symmetric encryption is still widely used due to its efficiency and speed.
Let's recap: Symmetric encryption has drawbacks because it requires access to the same key, compromises the key can make all data vulnerable, and 

## Cipher Math problems

In [26]:
result = generate_response(crypto_dataset[45]['question'],base_model)
print(f"Input:\n{crypto_dataset[45]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[45]['answer']}")

Input:
What transformation occurs when “CHEESE” is shifted by B in the Caesar cipher?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
What transformation occurs when “CHEESE” is shifted by B in the Caesar cipher?

### Response: Let's think step by step.
In the Caesar cipher, each letter is shifted by a fixed number of positions, usually by 3.
So, when "CHEESE" is shifted by B, each letter is shifted by B positions.
The letter "C" becomes "E", the letter "H" becomes "I", the letter "E" becomes "L", and so on.
After shifting by B positions, the word "CHEESE" becomes "EILIG".
Therefore, the transformation that occurs is the word "CHEESE" becomes "EILIG".

## Explaination
The Caesar cipher is a simple substitution cipher where each letter is shifted by a fixed number of positions in the alphabet. For example, if we shift each letter by 3 positions, the letter "A" becomes "D", the letter "B" b

In [27]:
result = generate_response(crypto_dataset[45]['question'],ft_model)
print(f"Input:\n{crypto_dataset[45]["question"]}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[45]['answer']}")

Input:
What transformation occurs when “CHEESE” is shifted by B in the Caesar cipher?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
What transformation occurs when “CHEESE” is shifted by B in the Caesar cipher?

### Response: Let's think step by step.
The letter “E” is the 5th letter in the alphabet. Shifting it by B places the letter “H” into the first position, “J” into the second position, and “L” into the third position. The rest of the alphabet is filled in this manner. As a result, the transformation of “CHEESE” is “EGHJMS”. 

### Impact of Answer Choice Changes

### No Change
Ground Truth 
 When “CHEESE” is shifted by B in the Caesar cipher, the transformation is “EJGGUG”.


In [28]:
result = generate_response(crypto_dataset[49]['question'],ft_model)
print(f"Input:\n{crypto_dataset[49]['question']}\n")
print(f"Generated Response \n {result[0]['generated_text'].split("The answer is")[0]}")
print(f"Ground Truth \n {crypto_dataset[49]['answer']}")

Input:
What is the process for decrypting the message 'yurgooaeod' using a rail fence with a depth of 2?

Generated Response 
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

 ### Instruction:
What is the process for decrypting the message 'yurgooaeod' using a rail fence with a depth of 2?

### Response: Let's think step by step.
The message 'yurgooaeod' is encrypted using a rail fence with a depth of 2. The decryption process is performed by reversing the encryption process. The encryption key is 2, which means that the data is distributed in stripes between the rails. The message is split into stripes of two letters each. The order of the stripes is reversed, starting from the last stripe and moving toward the first stripe. The individual stripes are then read from left to right, forming the original plaintext message 'decrypt'.  'yurgooaeod' is the plaintext message encrypted using a rail fence with a depth of 2.  'yurgooaeo